# **Data Preparation and Reshaping for Marketing Campaign Dashboard**

This notebook supports the **Marketing Campaign Performance Dashboard** by handling data preparation tasks that Excel 2013 could not perform (such as reshaping data from wide to long format).  
We use **Python (pandas)** to clean and restructure subsets of the dataset for product category and channel analysis.

---

## 🔹 Step 1: Load Data
- We load the `Prep_Data` sheet from the Excel file. This contains the cleaned dataset after basic preprocessing (age, income, family size, etc.).


In [8]:
import pandas as pd

# Load data from the 'Prep_Data' sheet of the Excel file
df = pd.read_excel("Marketing Canpagine Performance.xlsx", sheet_name="Prep_Data")
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,Age,Age_Cleaned,Age_Group,Total_Spend,Total_Camps_Accepted,Marital_Cleaned,Family_Size,Edu_Cleaned,Income_Cleaned,Income_Group
0,1826,1970,Graduation,Divorced,84835.0,0,0,2014-06-16,0,189,...,44,44,40-50,1190,1,Previously Married,1,Graduate,84835,High
1,1,1961,Graduation,Single,57091.0,0,0,2014-06-15,0,464,...,53,53,50+,577,2,Single,1,Graduate,57091,Medium
2,10476,1958,Graduation,Married,67267.0,0,1,2014-05-13,0,134,...,56,56,50+,251,0,In Relationship,3,Graduate,67267,Medium
3,1386,1967,Graduation,Together,32474.0,1,1,2014-05-11,0,10,...,47,47,40-50,11,0,In Relationship,4,Graduate,32474,Low
4,5371,1989,Graduation,Single,21474.0,1,0,2014-04-08,0,6,...,25,25,<30,91,2,Single,2,Graduate,21474,Low


## 🔹 Step 2: Create Buyer Flags for Each Product Category
- The dataset has spending columns for each product (`MntWines`, `MntFruits`, etc.).  
- To simplify analysis, we create **binary buyer flags** (1 = customer bought from that category, 0 = otherwise).


In [9]:
# List of product categories
products = ["MntWines", "MntFruits", "MntMeatProducts", 
            "MntFishProducts", "MntSweetProducts", "MntGoldProds"]

# Create buyer flags
for p in products:
    flag_name = p.replace("Mnt", "") + "_Buyer"
    df[flag_name] = (df[p] > 0).astype(int)

df.head()


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,Family_Size,Edu_Cleaned,Income_Cleaned,Income_Group,Wines_Buyer,Fruits_Buyer,MeatProducts_Buyer,FishProducts_Buyer,SweetProducts_Buyer,GoldProds_Buyer
0,1826,1970,Graduation,Divorced,84835.0,0,0,2014-06-16,0,189,...,1,Graduate,84835,High,1,1,1,1,1,1
1,1,1961,Graduation,Single,57091.0,0,0,2014-06-15,0,464,...,1,Graduate,57091,Medium,1,1,1,1,0,1
2,10476,1958,Graduation,Married,67267.0,0,1,2014-05-13,0,134,...,3,Graduate,67267,Medium,1,1,1,1,1,1
3,1386,1967,Graduation,Together,32474.0,1,1,2014-05-11,0,10,...,4,Graduate,32474,Low,1,0,1,0,0,0
4,5371,1989,Graduation,Single,21474.0,1,0,2014-04-08,0,6,...,2,Graduate,21474,Low,1,1,1,1,0,1


## 🔹 Step 3: Reshape Product Categories into Long Format
- Excel 2013 lacks Power Query/unpivot features.  
- We use `pandas.melt()` to convert wide product buyer flags into **long format**, making it easier to analyze campaign response rates by product category in pivot tables.


In [10]:
# Keep only ID, Response, and buyer flags
buyer_flags = [c for c in df.columns if c.endswith("_Buyer")]
reshaped = df.melt(
    id_vars=["ID", "Response"],
    value_vars=buyer_flags,
    var_name="ProductCategory",
    value_name="IsBuyer"
)
reshaped.head()

,ID,Response,ProductCategory,IsBuyer
0,1826,1,Wines_Buyer,1
1,1,1,Wines_Buyer,1
2,10476,0,Wines_Buyer,1
3,1386,0,Wines_Buyer,1
4,5371,1,Wines_Buyer,1


## 🔹 Step 4: Clean Product Category Names
- We remove the `_Buyer` suffix from column names for readability.


In [11]:
# Clean category names
reshaped["ProductCategory"] = reshaped["ProductCategory"].str.replace("_Buyer", "")
reshaped.head()

,ID,Response,ProductCategory,IsBuyer
0,1826,1,Wines,1
1,1,1,Wines,1
2,10476,0,Wines,1
3,1386,0,Wines,1
4,5371,1,Wines,1


## 🔹 Step 5: Export Reshaped Product Data
- The reshaped product dataset is saved as `Reshaped_Data.xlsx`.  
- This file is later used in Excel for pivot tables and dashboard visuals (Response by Product Category).


In [12]:
reshaped.to_excel("Reshaped_Data.xlsx", index=False)


---

## 🔹 Step 6: Reshape Channel Purchases into Long Format
- Similar to product categories, channel purchase columns (`NumWebPurchases`, `NumCatalogPurchases`, `NumStorePurchases`) are reshaped into **long format**.  
- This structure allows us to directly compare average purchases of responders vs non-responders across channels.


In [13]:
channels = ["NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases"]

df_long = pd.melt(
    df2,
    id_vars=["ID", "Response"],
    value_vars=channels,
    var_name="Channels",
    value_name="Purchases"
)

df_long.head()

,ID,Response,Channels,Purchases
0,1826,1,NumWebPurchases,4
1,1,1,NumWebPurchases,7
2,10476,0,NumWebPurchases,3
3,1386,0,NumWebPurchases,1
4,5371,1,NumWebPurchases,3


## 🔹 Step 7: Export Channel Data
- The reshaped channel dataset is saved as `Marketing_Channel.xlsx` with a dedicated `Channel_Data` sheet.  
- This data powers the **Avg Purchases by Channel** chart in the Excel dashboard.


In [14]:
df_long.to_excel("Marketing_Channel.xlsx", index=False , sheet_name="Channel_Data")

---

## ✅ Outcome
- Created **buyer flags** for all product categories.  
- Reshaped product data and channel data into **long format** using pandas.  
- Exported both datasets back to Excel for further pivot table analysis.  

This workflow demonstrates how **Python and Excel** can complement each other: Python for efficient data manipulation, Excel for business-friendly visualization and dashboarding.
